In [1]:
# SUP GUYS IT'S YA GIRL LET'S DO THIS SHIT
# here I am importing things 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
#from mpl_toolkits.basemap import Basemap  #Tom  doesn't have this. How do you install it?

%matplotlib inline

In [2]:
test = pd.read_csv('./assets/test.csv')
train = pd.read_csv('./assets/train.csv')
spray = pd.read_csv('./assets/spray.csv')
weather = pd.read_csv('./assets/weather.csv')

### Cleaning up the 'Weather' data

In [3]:
weather.head() #looks like station 2 has a lot of missing data

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,...,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
0,1,2007-05-01,83,50,67,14,51,56,0,2,...,,0,M,0.0,0.00,29.10,29.82,1.7,27,9.2
1,2,2007-05-01,84,52,68,M,51,57,0,3,...,,M,M,M,0.00,29.18,29.82,2.7,25,9.6
2,1,2007-05-02,59,42,51,-3,42,47,14,0,...,BR,0,M,0.0,0.00,29.38,30.09,13.0,4,13.4
3,2,2007-05-02,60,43,52,M,42,47,13,0,...,BR HZ,M,M,M,0.00,29.44,30.08,13.3,2,13.4
4,1,2007-05-03,66,46,56,2,40,48,9,0,...,,0,M,0.0,0.00,29.39,30.12,11.7,7,11.9


In [4]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2944 entries, 0 to 2943
Data columns (total 22 columns):
Station        2944 non-null int64
Date           2944 non-null object
Tmax           2944 non-null int64
Tmin           2944 non-null int64
Tavg           2944 non-null object
Depart         2944 non-null object
DewPoint       2944 non-null int64
WetBulb        2944 non-null object
Heat           2944 non-null object
Cool           2944 non-null object
Sunrise        2944 non-null object
Sunset         2944 non-null object
CodeSum        2944 non-null object
Depth          2944 non-null object
Water1         2944 non-null object
SnowFall       2944 non-null object
PrecipTotal    2944 non-null object
StnPressure    2944 non-null object
SeaLevel       2944 non-null object
ResultSpeed    2944 non-null float64
ResultDir      2944 non-null int64
AvgSpeed       2944 non-null object
dtypes: float64(1), int64(5), object(16)
memory usage: 506.1+ KB


In [5]:
#not using the CodeSum in this iteration
weather = weather.drop('CodeSum', axis=1)

In [6]:
# here we are filling in the missing 'Tavg' data with the average 

#weather['Tavg'].fillna(((weather['Tmax'] + weather['Tmin']) / 2), inplace = True)

In [7]:
# here we will replace 'M' and 'T' with nan values (as they stand for missing and trace amounts) 

weather.replace('M', np.nan, inplace = True)
weather.replace('  T', np.nan, inplace = True) #Why are there two spaces before T?
weather.replace('-', np.nan, inplace = True)

In [8]:
#combine station 2 data with station 1 data
weather_stn1 = weather[weather['Station']==1]
weather_stn2 = weather[weather['Station']==2]
weather_stn1 = weather_stn1.drop('Station', axis=1)
weather_stn2 = weather_stn2.drop('Station', axis=1)
weather = weather_stn1.merge(weather_stn2, on='Date')

In [9]:
weather.head(10) # Checking to make sure it worked. There are a few NANs and .005s in there.

,Date,Tmax_x,Tmin_x,Tavg_x,Depart_x,DewPoint_x,WetBulb_x,Heat_x,Cool_x,Sunrise_x,...,Sunset_y,Depth_y,Water1_y,SnowFall_y,PrecipTotal_y,StnPressure_y,SeaLevel_y,ResultSpeed_y,ResultDir_y,AvgSpeed_y
0,2007-05-01,83,50,67,14,51,56,0,2,0448,...,NaN,NaN,NaN,NaN,0.00,29.18,29.82,2.7,25,9.6
1,2007-05-02,59,42,51,-3,42,47,14,0,0447,...,NaN,NaN,NaN,NaN,0.00,29.44,30.08,13.3,2,13.4
2,2007-05-03,66,46,56,2,40,48,9,0,0446,...,NaN,NaN,NaN,NaN,0.00,29.46,30.12,12.9,6,13.2
3,2007-05-04,66,49,58,4,41,50,7,0,0444,...,NaN,NaN,NaN,NaN,0.00,29.36,30.04,10.1,7,10.4
4,2007-05-05,66,53,60,5,38,49,5,0,0443,...,NaN,NaN,NaN,NaN,NaN,29.46,30.09,11.2,7,11.5
5,2007-05-06,68,49,59,4,30,46,6,0,0442,...,NaN,NaN,NaN,NaN,0.00,29.62,30.28,13.8,10,14.5
6,2007-05-07,83,47,65,10,41,54,0,0,0441,...,NaN,NaN,NaN,NaN,0.00,29.44,30.12,8.5,17,9.9
7,2007-05-08,82,54,68,12,58,62,0,3,0439,...,NaN,NaN,NaN,NaN,NaN,29.36,30.02,2.5,8,5.4
8,2007-05-09,77,61,69,13,59,63,0,4,0438,...,NaN,NaN,NaN,NaN,0.02,29.28,29.93,3.9,7,5.9
9,2007-05-10,84,56,70,14,52,60,0,5,0437,...,NaN,NaN,NaN,NaN,0.00,29.26,29.91,2.0,9,3.9


In [10]:
weather.isnull().sum()

Date                0
Tmax_x              0
Tmin_x              0
Tavg_x              0
Depart_x            0
DewPoint_x          0
WetBulb_x           3
Heat_x              0
Cool_x              0
Sunrise_x           0
Sunset_x            0
Depth_x             0
Water1_x         1472
SnowFall_x         12
PrecipTotal_x     163
StnPressure_x       2
SeaLevel_x          5
ResultSpeed_x       0
ResultDir_x         0
AvgSpeed_x          0
Tmax_y              0
Tmin_y              0
Tavg_y             11
Depart_y         1472
DewPoint_y          0
WetBulb_y           1
Heat_y             11
Cool_y             11
Sunrise_y        1472
Sunset_y         1472
Depth_y          1472
Water1_y         1472
SnowFall_y       1472
PrecipTotal_y     157
StnPressure_y       2
SeaLevel_y          4
ResultSpeed_y       0
ResultDir_y         0
AvgSpeed_y          3
dtype: int64

In [11]:
# Getting a list of all of the columns with null values
nulls = pd.DataFrame(weather.isnull().sum())
weather_nulls = nulls[nulls[0]>0].index

In [12]:
# deleting the columns that give little to no information

weather.drop(columns = weather_nulls, inplace = True)

In [13]:
# making the columns with null values easier to read
# What null columns are left to deal with

null_columns = weather.columns[weather.isnull().any()]
weather[null_columns].isnull().sum()

Series([], dtype: float64)

In [14]:
print('Null Rows: ', weather[weather.isnull().any(axis=1)][null_columns].shape[0])
print('Total Rows: ', weather.shape[0])


Null Rows:  0
Total Rows:  1472


In [15]:
objs = []
for col in weather.columns:
    if weather[col].dtypes == 'O':
        objs.append(col)
        
weather[objs].dtypes

Date          object
Tavg_x        object
Depart_x      object
Heat_x        object
Cool_x        object
Sunrise_x     object
Sunset_x      object
Depth_x       object
AvgSpeed_x    object
dtype: object

In [16]:
floats = ['AvgSpeed_x']
ints = ['Tavg_x', 'Depart_x', 'Heat_x', 'Cool_x', 'Depth_x']
weather[floats] = weather[floats].astype(float)
weather[ints] = weather[ints].astype(int)
#print(floats)
#print(ints)

In [17]:
#weather.drop(columns = ['Heat', 'Cool'], inplace = True)

## Cleaning / EDA for 'Train' and 'Test'

In [18]:
# checking out the first five rows of the test data

test.head()

,Id,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy
0,1,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
1,2,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
2,3,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
3,4,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX SALINARIUS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
4,5,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX TERRITANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9


In [19]:
train.head()

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
2,2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX RESTUANS,62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,1,0
3,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX PIPIENS/RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,1,0
4,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,4,0


In [20]:
# wanted to see how the counts of mosquitos and the infected mosquito counts
# related

train.sort_values(['WnvPresent'], ascending = False).head(10)

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent
2920,2007-09-05,"2200 West 113th Street, Chicago, IL 60643, USA",CULEX PIPIENS,22,W 113TH ST,T086,"2200 W 113TH ST, Chicago, IL",41.688324,-87.676709,8,13,1
7326,2011-08-12,"Ohare Court, Bensenville, IL 60106, USA",CULEX PIPIENS/RESTUANS,10,W OHARE,T903,"1000 W OHARE, Chicago, IL",41.957799,-87.930995,5,8,1
9806,2013-08-22,"3700 South Pulaski Road, Chicago, IL 60623, USA",CULEX PIPIENS/RESTUANS,37,S PULASKI RD,T063,"3700 S PULASKI RD, Chicago, IL",41.825610,-87.726549,9,6,1
9799,2013-08-22,"5000 South Union Avenue, Chicago, IL 60609, USA",CULEX PIPIENS/RESTUANS,50,S UNION AVE,T082,"5000 S UNION AVE, Chicago, IL",41.803423,-87.642984,8,15,1
2343,2007-08-21,"5200 South Nordica Avenue, Chicago, IL 60638, USA",CULEX PIPIENS,52,S NORDICA,T070,"5200 S NORDICA, Chicago, IL",41.797616,-87.797894,8,10,1
9788,2013-08-22,"6700 South Kedzie Avenue, Chicago, IL 60629, USA",CULEX PIPIENS,67,S KEDZIE AVE,T066,"6700 S KEDZIE AVE, Chicago, IL",41.771660,-87.703047,8,14,1
9787,2013-08-22,"6700 South Kedzie Avenue, Chicago, IL 60629, USA",CULEX RESTUANS,67,S KEDZIE AVE,T066,"6700 S KEDZIE AVE, Chicago, IL",41.771660,-87.703047,8,4,1
2342,2007-08-21,"5200 South Nordica Avenue, Chicago, IL 60638, USA",CULEX PIPIENS/RESTUANS,52,S NORDICA,T070,"5200 S NORDICA, Chicago, IL",41.797616,-87.797894,8,13,1
7331,2011-08-12,"5100 North Mont Clare Avenue, Chicago, IL 6065...",CULEX RESTUANS,51,N MONT CLARE AVE,T223,"5100 N MONT CLARE AVE, Chicago, IL",41.973845,-87.805059,9,3,1
9782,2013-08-22,"2200 West 51st Street, Chicago, IL 60609, USA",CULEX RESTUANS,22,W 51ST ST,T061,"2200 W 51ST ST, Chicago, IL",41.801179,-87.679447,8,1,1


In [21]:
def create_month(x):
    return x.split('-')[1]

def create_day(x):
    return x.split('-')[2]

def create_year(x):
    return x.split('-')[0]

In [22]:
# creating new columns to break out the month, day, and year 

train['month'] = train['Date'].apply(create_month)
train['day'] = train['Date'].apply(create_day)
train['year'] = train['Date'].apply(create_year)
test['month'] = test['Date'].apply(create_month)
test['day'] = test['Date'].apply(create_day)

In [24]:
# merge the relevant weather data with the training and testing data

train = train.merge(weather, on='Date')
test = test.merge(weather, on='Date')


# drop the redundant date column
train = train.drop(['Date'], axis = 1)
test = test.drop(['Date'], axis = 1)

In [25]:
# Convert categorical data to numbers
lbl = LabelEncoder()
lbl.fit(list(train['Species'].values) + list(test['Species'].values))
train['Species'] = lbl.transform(train['Species'].values)
test['Species'] = lbl.transform(test['Species'].values)

In [26]:
lbl.fit(list(train['Street'].values) + list(test['Street'].values))
train['Street'] = lbl.transform(train['Street'].values)
test['Street'] = lbl.transform(test['Street'].values)

In [27]:
lbl.fit(list(train['Trap'].values) + list(test['Trap'].values))
train['Trap'] = lbl.transform(train['Trap'].values)
test['Trap'] = lbl.transform(test['Trap'].values)

### Now to the "Spray" data

In [28]:
spray.head()

,Date,Time,Latitude,Longitude
0,2011-08-29,6:56:58 PM,42.391623,-88.089163
1,2011-08-29,6:57:08 PM,42.391348,-88.089163
2,2011-08-29,6:57:18 PM,42.391022,-88.089157
3,2011-08-29,6:57:28 PM,42.390637,-88.089158
4,2011-08-29,6:57:38 PM,42.390410,-88.088858


In [29]:
spray.isnull().sum()

Date           0
Time         584
Latitude       0
Longitude      0
dtype: int64

In [30]:
# drop time column at the moment. Might use it in a later iteration 

spray.drop(columns = ['Time'], inplace = True)

In [31]:
# check to make sure it worked

spray.head()

,Date,Latitude,Longitude
0,2011-08-29,42.391623,-88.089163
1,2011-08-29,42.391348,-88.089163
2,2011-08-29,42.391022,-88.089157
3,2011-08-29,42.390637,-88.089158
4,2011-08-29,42.390410,-88.088858


In [32]:
# THIS IS A TEST AND IS NOT GOOD, TK WAS WORKING ON BASEMAP AS WELL. LEAVING THIS IN INCASE ANYONE
# WANTS TO WORK ON IT

# plt.figure(figsize=(8, 8))
# earth = Basemap(projection = 'merc',
#                 lon_0 = -87.6298,
#                 lat_0 = 41.8781,
#                 llcrnrlon=-85, 
#                 llcrnrlat=39,
#                 urcrnrlon=-89, 
#                 urcrnrlat=45,
#                 resolution = 'h'
#                )
# earth.bluemarble(alpha=0.42)
# earth.drawcoastlines(color='grey', linewidth=1)
# #earth.drawcounties(color = 'grey', linewidth=1)
# plt.scatter(spray['Longitude'], spray['Latitude'], 
#             c='red',alpha=.5)
# #plt.figsize()
# plt.xlabel("Test");

## Setting the columns to their correct types


In [33]:
train.dtypes

Address                    object
Species                     int64
Block                       int64
Street                      int64
Trap                        int64
AddressNumberAndStreet     object
Latitude                  float64
Longitude                 float64
AddressAccuracy             int64
NumMosquitos                int64
WnvPresent                  int64
month                      object
day                        object
year                       object
Tmax_x                      int64
Tmin_x                      int64
Tavg_x                      int64
Depart_x                    int64
DewPoint_x                  int64
Heat_x                      int64
Cool_x                      int64
Sunrise_x                  object
Sunset_x                   object
Depth_x                     int64
ResultSpeed_x             float64
ResultDir_x                 int64
AvgSpeed_x                float64
Tmax_y                      int64
Tmin_y                      int64
DewPoint_y    

In [34]:
train.head(1) 

,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,...,Sunset_x,Depth_x,ResultSpeed_x,ResultDir_x,AvgSpeed_x,Tmax_y,Tmin_y,DewPoint_y,ResultSpeed_y,ResultDir_y
0,"4100 North Oak Park Avenue, Chicago, IL 60634,...",2,41,36,1,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,1,...,1917,0,5.8,18,6.5,88,65,59,5.8,16


In [45]:
ints = ['day', 'month', 'year']
train[ints] = train[ints].astype(int)


In [35]:
test.dtypes #these types look right

Id                          int64
Address                    object
Species                     int64
Block                       int64
Street                      int64
Trap                        int64
AddressNumberAndStreet     object
Latitude                  float64
Longitude                 float64
AddressAccuracy             int64
month                      object
day                        object
Tmax_x                      int64
Tmin_x                      int64
Tavg_x                      int64
Depart_x                    int64
DewPoint_x                  int64
Heat_x                      int64
Cool_x                      int64
Sunrise_x                  object
Sunset_x                   object
Depth_x                     int64
ResultSpeed_x             float64
ResultDir_x                 int64
AvgSpeed_x                float64
Tmax_y                      int64
Tmin_y                      int64
DewPoint_y                  int64
ResultSpeed_y             float64
ResultDir_y   

In [46]:
test.head(1)

,Id,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,...,Sunset_x,Depth_x,ResultSpeed_x,ResultDir_x,AvgSpeed_x,Tmax_y,Tmin_y,DewPoint_y,ResultSpeed_y,ResultDir_y
0,1,"4100 North Oak Park Avenue, Chicago, IL 60634,...",2,41,36,1,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,...,1926,0,8.9,18,10.0,86,66,55,9.4,18


In [47]:
ints = ['day', 'month']
test[ints] = test[ints].astype(int)


Spray

In [48]:
spray.dtypes

Date          object
Latitude     float64
Longitude    float64
dtype: object

In [49]:
test.isnull().sum().sum()

0

In [50]:
train.isnull().sum().sum()

0

In [51]:
weather.isnull().sum().sum()

0

In [52]:
spray.isnull().sum().sum()

0

## Save
Save the cleaned files as new csvs to use in other notebooks.

In [53]:
train.to_csv('./assets/train_cleaned.csv')
test.to_csv('./assets/test_cleaned.csv')
spray.to_csv('./assets/spray_cleaned.csv')
weather.to_csv('./assets/weather_cleaned.csv')
